In [12]:
import pyodbc
import pandas as pd

# Define your connection string
conn_str = (           
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=NGUYEN;"
    "DATABASE=CompanyX;"
    "Trusted_Connection=yes;"
    # "UID=YourUsername;"
    # "PWD=YourPassword;"
)

# Establish connection
conn = pyodbc.connect(conn_str)


In [13]:
# Write SQL query
query1 = "SELECT * FROM DimCustomer"

# Execute query and load data into a DataFrame
DimCustomer = pd.read_sql(query1, conn)

# Write SQL query
query2 = "SELECT * FROM DimSalesTerritory"

# Execute query and load data into a DataFrame
DimSalesTerritory = pd.read_sql(query2, conn)

# Write SQL query
query3 = "SELECT * FROM DimProduct"

# Execute query and load data into a DataFrame
DimProduct = pd.read_sql(query3, conn)

# Write SQL query
query4 = "SELECT * FROM DimDate"

# Execute query and load data into a DataFrame
DimDate = pd.read_sql(query4, conn)

# Write SQL query
query5 = "SELECT * FROM DimGeography"

# Execute query and load data into a DataFrame
DimGeography = pd.read_sql(query5, conn)

# Write SQL query
query6 = "SELECT * FROM DimPromotion"

# Execute query and load data into a DataFrame
DimPromotion = pd.read_sql(query6, conn)

# Write SQL query
query7 = "SELECT * FROM FactSales"

# Execute query and load data into a DataFrame
FactSales = pd.read_sql(query7, conn)


C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\2241041406.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimCustomer = pd.read_sql(query1, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\2241041406.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimSalesTerritory = pd.read_sql(query2, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\2241041406.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  DimProduct = pd.read_sql(query3, conn)
C:\Users\Khoi Nguyen\AppData\Local\Temp\ipyke

In [14]:
DimProduct['ProductCategoryName'] = DimProduct['ProductCategoryName'].fillna('Other')
DimProduct['ProductCategoryName'] = DimProduct['ProductCategoryName'].astype('category')
DimProduct = DimProduct[DimProduct['ProductCategoryName'] != 'Other']

In [15]:
DimProduct.head()

,Name,ProductNumber,ReorderPoint,SafetyStockLevel,Color,ListPrice,Style,FinishedGoodsFlag,MakeFlag,DaysToManufacture,ProductCategoryName,ProductKey,ModifiedDate
209,"HL Road Frame - Black, 58",FR-R92B-58,375,500,Black,1431.50,U,True,True,1,Components,680,2014-02-08 10:01:36.827
210,"HL Road Frame - Red, 58",FR-R92R-58,375,500,Red,1431.50,U,True,True,1,Components,706,2014-02-08 10:01:36.827
211,"Sport-100 Helmet, Red",HL-U509-R,3,4,Red,34.99,None,True,False,0,Accessories,707,2014-02-08 10:01:36.827
212,"Sport-100 Helmet, Black",HL-U509,3,4,Black,34.99,None,True,False,0,Accessories,708,2014-02-08 10:01:36.827
213,"Mountain Bike Socks, M",SO-B909-M,3,4,White,9.50,U,True,False,0,Clothing,709,2014-02-08 10:01:36.827


In [16]:
FactSales.head()

,SalesOrderNumber,SalesLineNumber,RevisionNumber,OrderQty,UnitPrice,CarrierTrackingNumber,SalesAmount,ProductID,UnitPriceDiscount,PurchaseOrderNumber,OrderDate,TaxAmt,Freight,SpecialOfferID,CustomerID,TerritoryID,StandardCost,ModifiedDate,TotalProductCost
0,43659,1,8,1.0,2024.9940,4911-403C-98,2024.9940,776,0.0,PO522145787,2011-05-31,161.999520,50.62485,1,29825,5,1898.0944,2011-05-31,1898.0944
1,43659,2,8,4.0,20.1865,4911-403C-98,80.7460,711,0.0,PO522145787,2011-05-31,6.459680,2.01865,1,29825,5,13.0863,2011-05-31,52.3452
2,43659,3,8,6.0,5.7000,4911-403C-98,34.2000,709,0.0,PO522145787,2011-05-31,2.736000,0.85500,1,29825,5,3.3963,2011-05-31,20.3778
3,43659,4,8,1.0,28.8404,4911-403C-98,28.8404,716,0.0,PO522145787,2011-05-31,2.307232,0.72101,1,29825,5,38.4923,2011-05-31,38.4923
4,43659,5,8,3.0,28.8404,4911-403C-98,86.5212,714,0.0,PO522145787,2011-05-31,6.921696,2.16303,1,29825,5,38.4923,2011-05-31,115.4769


In [17]:
FactSales = pd.merge(FactSales, DimProduct[['ProductKey', 'ProductCategoryName']],
                     left_on='ProductID',
                     right_on='ProductKey',
                     how='left')

FactSales.drop(columns='ProductKey', inplace=True)

### Total Revenue per day per sales territory

In [18]:
sales_per_day_per_product = (
    FactSales.groupby(["OrderDate","ProductCategoryName", "TerritoryID"])["SalesAmount"]
    .sum()
    .reset_index())

# sales_per_day_per_territory = sales_per_day_per_territory.sort_values(by="OrderDate")

sales_per_day_per_product.head()

C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\2009132387.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  FactSales.groupby(["OrderDate","ProductCategoryName", "TerritoryID"])["SalesAmount"]


,OrderDate,ProductCategoryName,TerritoryID,SalesAmount
0,2011-05-31,Accessories,1,222.0515
1,2011-05-31,Accessories,2,141.3055
2,2011-05-31,Accessories,3,121.1190
3,2011-05-31,Accessories,4,40.3730
4,2011-05-31,Accessories,5,302.7975


### Average Profit Margin per day per sales territory

In [19]:
FactSales['Average Profit Margin'] = ((FactSales['SalesAmount'] - FactSales['TotalProductCost']) / FactSales['TotalProductCost']) * 100

average_profit_per_day_per_product = (
    FactSales.groupby(["OrderDate", "ProductCategoryName", "TerritoryID"])[["Average Profit Margin", "OrderQty", "UnitPrice", "UnitPriceDiscount", "TaxAmt", "Freight"]]
    .sum()
    .reset_index()
)
average_profit_per_day_per_product.head()

C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\1035769447.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  FactSales.groupby(["OrderDate", "ProductCategoryName", "TerritoryID"])[["Average Profit Margin", "OrderQty", "UnitPrice", "UnitPriceDiscount", "TaxAmt", "Freight"]]


,OrderDate,ProductCategoryName,TerritoryID,Average Profit Margin,OrderQty,UnitPrice,UnitPriceDiscount,TaxAmt,Freight
0,2011-05-31,Accessories,1,379.797192,11.0,141.3055,0.0,17.76412,5.551285
1,2011-05-31,Accessories,2,162.770225,7.0,60.5595,0.0,11.30444,3.532637
2,2011-05-31,Accessories,3,108.513484,6.0,40.3730,0.0,9.68952,3.027974
3,2011-05-31,Accessories,4,-600.000000,18.0,0.0000,0.0,3.22984,1.009325
4,2011-05-31,Accessories,5,325.540451,15.0,121.1190,0.0,24.22380,7.569936


### Average revenue of All Customer per Sales Territory

In [20]:
result = (
    FactSales.groupby(["OrderDate","ProductCategoryName", "TerritoryID"])
    .agg(
        TotalSalesAmount=("SalesAmount", "sum"),
        TotalCustomers=("CustomerID", "nunique")
    )
    .reset_index()
)

# Calculate the ratio: SalesAmount / Total Customers
result["Average_Revenue_Per_User"] = result["TotalSalesAmount"] / result["TotalCustomers"]

columns_to_drop = ["TotalSalesAmount", "TotalCustomers"]
result = result.drop(columns_to_drop, axis=1)

result.head()

C:\Users\Khoi Nguyen\AppData\Local\Temp\ipykernel_26092\2802390429.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  FactSales.groupby(["OrderDate","ProductCategoryName", "TerritoryID"])


,OrderDate,ProductCategoryName,TerritoryID,Average_Revenue_Per_User
0,2011-05-31,Accessories,1,74.017167
1,2011-05-31,Accessories,2,141.305500
2,2011-05-31,Accessories,3,121.119000
3,2011-05-31,Accessories,4,20.186500
4,2011-05-31,Accessories,5,100.932500


In [ ]:
merged_data = pd.merge(
    sales_per_day_per_product,
    average_profit_per_day_per_product,
    on=["OrderDate", "ProductCategoryName", "TerritoryID"],
    how="inner"
)

merged_data = pd.merge(
    merged_data,
    result,
    on=["OrderDate", "ProductCategoryName", "TerritoryID"],
    how="inner"
)

,OrderDate,ProductCategoryName,TerritoryID,SalesAmount,Average Profit Margin,OrderQty,UnitPrice,UnitPriceDiscount,TaxAmt,Freight,Average_Revenue_Per_User
0,2011-05-31,Accessories,1,222.0515,379.797192,11.0,141.3055,0.0,17.76412,5.551285,74.017167
1,2011-05-31,Accessories,2,141.3055,162.770225,7.0,60.5595,0.0,11.30444,3.532637,141.305500
2,2011-05-31,Accessories,3,121.1190,108.513484,6.0,40.3730,0.0,9.68952,3.027974,121.119000
3,2011-05-31,Accessories,4,40.3730,-600.000000,18.0,0.0000,0.0,3.22984,1.009325,20.186500
4,2011-05-31,Accessories,5,302.7975,325.540451,15.0,121.1190,0.0,24.22380,7.569936,100.932500


In [26]:
territory_mapping = {
    1: "Northwest, US",
    2: "Northeast, US",
    3: "Central, US",
    4: "Southwest, US",
    5: "Southeast, US",
    6: "Canada",
    7: "France",
    8: "Germany",
    9: "Australia",
    10: " United Kingdom" 
}

merged_data['TerritoryName'] = merged_data['TerritoryID'].map(territory_mapping)

merged_data.insert(3, 'TerritoryName', merged_data.pop('TerritoryName'))

In [27]:
merged_data.head()

,OrderDate,ProductCategoryName,TerritoryID,TerritoryName,SalesAmount,Average Profit Margin,OrderQty,UnitPrice,UnitPriceDiscount,TaxAmt,Freight,Average_Revenue_Per_User
0,2011-05-31,Accessories,1,"Northwest, US",222.0515,379.797192,11.0,141.3055,0.0,17.76412,5.551285,74.017167
1,2011-05-31,Accessories,2,"Northeast, US",141.3055,162.770225,7.0,60.5595,0.0,11.30444,3.532637,141.305500
2,2011-05-31,Accessories,3,"Central, US",121.1190,108.513484,6.0,40.3730,0.0,9.68952,3.027974,121.119000
3,2011-05-31,Accessories,4,"Southwest, US",40.3730,-600.000000,18.0,0.0000,0.0,3.22984,1.009325,20.186500
4,2011-05-31,Accessories,5,"Southeast, US",302.7975,325.540451,15.0,121.1190,0.0,24.22380,7.569936,100.932500


In [ ]:
merged_data = merged_data.dropna()

In [30]:
from sqlalchemy import create_engine

# Define your connection string for SQLAlchemy
conn_str = (
    "mssql+pyodbc://@NGUYEN/CompanyX?driver=ODBC+Driver+17+for+SQL+Server"
)

# Create SQLAlchemy engine
engine = create_engine(conn_str)

# Assuming DimCustomer is the DataFrame you want to export
# Specify the target table name
table_name = "Dim_Data"

# Export DataFrame to SQL Server
merged_data.to_sql(
    name=table_name, 
    con=engine, 
    if_exists='replace',  # Options: 'fail', 'replace', 'append'
    index=False           # Whether to write the DataFrame index
)

print(f"Data exported to table '{table_name}' successfully!")


Data exported to table 'Dim_Data' successfully!


In [23]:
# merged_data.to_csv('metrics_per_product.csv', index=False)